<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Overview-of-the-Approach-of-this-Project:" data-toc-modified-id="Overview-of-the-Approach-of-this-Project:-0.1"><strong><em>Overview of the Approach of this Project:</em></strong></a></span></li><li><span><a href="#1.-Extracting-Tables" data-toc-modified-id="1.-Extracting-Tables-0.2">1. Extracting Tables</a></span><ul class="toc-item"><li><span><a href="#Format-of-tables-in-the-Source-HTML-of-the-Web-page" data-toc-modified-id="Format-of-tables-in-the-Source-HTML-of-the-Web-page-0.2.1">Format of tables in the Source HTML of the Web page</a></span></li><li><span><a href="#Extracting-Table-1" data-toc-modified-id="Extracting-Table-1-0.2.2">Extracting Table 1</a></span></li></ul></li><li><span><a href="#Extracting-Table-2" data-toc-modified-id="Extracting-Table-2-0.3">Extracting Table 2</a></span></li><li><span><a href="#2.-Extracting-all-tables-in-the-HTML-Source-using-Functions" data-toc-modified-id="2.-Extracting-all-tables-in-the-HTML-Source-using-Functions-0.4">2. Extracting all tables in the HTML Source using Functions</a></span></li><li><span><a href="#3.-Data-Cleansing-and-Transformations" data-toc-modified-id="3.-Data-Cleansing-and-Transformations-0.5">3. Data Cleansing and Transformations</a></span><ul class="toc-item"><li><span><a href="#Transformation-1:-Creating-new-Dataframes-by-selecting-the-required-columns" data-toc-modified-id="Transformation-1:-Creating-new-Dataframes-by-selecting-the-required-columns-0.5.1">Transformation 1: Creating new Dataframes by selecting the required columns</a></span></li><li><span><a href="#Transformation-2:-Renaming-columns" data-toc-modified-id="Transformation-2:-Renaming-columns-0.5.2">Transformation 2: Renaming columns</a></span></li><li><span><a href="#Transformation-3:-Converting-data-to-Upper-case" data-toc-modified-id="Transformation-3:-Converting-data-to-Upper-case-0.5.3">Transformation 3: Converting data to Upper case</a></span></li><li><span><a href="#Transformation-4:-Remove-comma-seperator-and-convert-to-Integer" data-toc-modified-id="Transformation-4:-Remove-comma-seperator-and-convert-to-Integer-0.5.4">Transformation 4: Remove comma seperator and convert to Integer</a></span></li><li><span><a href="#Transformation-5:-Checking-for-Nulls" data-toc-modified-id="Transformation-5:-Checking-for-Nulls-0.5.5">Transformation 5: Checking for Nulls</a></span></li><li><span><a href="#Transformation-6:-Merging-Dataframes-into-1" data-toc-modified-id="Transformation-6:-Merging-Dataframes-into-1-0.5.6">Transformation 6: Merging Dataframes into 1</a></span></li><li><span><a href="#Transformation-7:-Drop-redundant-columns,-Outliers-and-Rename-other-columns-in-Merged-Dataframe" data-toc-modified-id="Transformation-7:-Drop-redundant-columns,-Outliers-and-Rename-other-columns-in-Merged-Dataframe-0.5.7">Transformation 7: Drop redundant columns, Outliers and Rename other columns in Merged Dataframe</a></span></li><li><span><a href="#Transformation-8:-Adding-New-columns" data-toc-modified-id="Transformation-8:-Adding-New-columns-0.5.8">Transformation 8: Adding New columns</a></span></li><li><span><a href="#Transformation-9:-GroupBy-and-Sort-Transformations" data-toc-modified-id="Transformation-9:-GroupBy-and-Sort-Transformations-0.5.9">Transformation 9: GroupBy and Sort Transformations</a></span></li><li><span><a href="#Transformation-10:-Stack-and-Unstack-Transformations" data-toc-modified-id="Transformation-10:-Stack-and-Unstack-Transformations-0.5.10">Transformation 10: Stack and Unstack Transformations</a></span></li><li><span><a href="#Transformation-11:-Pivot-&amp;-Melt-Transformations" data-toc-modified-id="Transformation-11:-Pivot-&amp;-Melt-Transformations-0.5.11">Transformation 11: Pivot &amp; Melt Transformations</a></span></li></ul></li><li><span><a href="#4.-Printing-the-Final-dataframe" data-toc-modified-id="4.-Printing-the-Final-dataframe-0.6">4. Printing the Final dataframe</a></span></li><li><span><a href="#5.Ethical-Implications" data-toc-modified-id="5.Ethical-Implications-0.7">5.Ethical Implications</a></span></li></ul></li><li><span><a href="#" data-toc-modified-id="-1"></a></span></li></ul></div>

# Cleaning/Formatting Website data

In [ ]:
# Importing Beautiful soup
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import os

In [ ]:
# Setting global options for the notebook such as maxrows
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_rows", 50)
import warnings
warnings.filterwarnings('ignore')

### ***Overview of the Approach of this Project:*** 
##### 1. In this project, the data from Website https://www.iihs.org/topics/fatality-statistics/detail/state-by-state#crash-types is sourced , cleansed by applying various transformations which will be used for plotting visualizations(later in the final project)
##### 2. There are 6 tables in the website HTML source. Using Beautiful soup for web scraping, data from all 6 tables are extracted and Dataframes are built on the data. As all the tables have similar structure, functions are used to extract data from all 6 tables.
##### 3. After the Headings and the data from the tables are extracted, Dataframes are created. 
##### 4. Out of the 6 tables, 5 tables will be used in this Project. The required columns are selected from the Dataframes.
##### 5. The data is then cleansed by removing duplicates, checking for Nulls, Converting to Upper case, etc and finally all the 5 Dataframes are joined together using common columns.
##### 6. The data is then joined with the US states data to include additional details such as Region, Division, etc.
##### 7. Finally multiple transformations such as GroupBy, Sort, Pivot are applied to the final dataframe which will be used in the final Milestone project for Visualization.


### 1. Extracting Tables

In [ ]:
# Extracting data from the Website using Beautiful Soup
url = "https://www.iihs.org/topics/fatality-statistics/detail/state-by-state#crash-types"
# Using Requests library to pull the Source HTML from the website
req = requests.get(url)
# Creating Beautiful Soup Object
soup = BeautifulSoup(req.content, 'html.parser')
# Printing the HTML
#print(soup.prettify())

#### Format of tables in the Source HTML of the Web page

##### 1. There are about 6 tables in the HTML Source. All of them are contained in a div tag and they all share the same class name- "table fatality-table is-striped is-bordered is-hoverable is-fullwidth". Hence the div tags with this class name is found and tables within the div tag are searched. 
##### 2. All the tables have similar structure where the Title and the Header data are contained within the thead section and the table data within the tbody section. 
##### 3. To extract the table title and Headers, the thead is parsed and the "th" within the "tr" are then parsed.
##### 4. Similarly to extract the table data, the tbody is parsed and the "td" within the "th" are parsed.
##### 5. For the tables 2 to 6, the statistics along with the Percentage information is present in the tables. The Percentage data is skipped while reading the table data.
##### 6. As all the 6 tables use similar table structure, resuable function is used to extract the data.

In [ ]:
# Counting the number of html tables in the webpage
fatalities_tables=soup.find_all('table')
print(f"There are {len(fatalities_tables)} tables in the Web page")

In [ ]:
# Counting the number of html tables by checking the <div> with the desired class name
fatalities_divs=soup.find_all('div', class_="table fatality-table is-striped is-bordered is-hoverable is-fullwidth")
# Checking the number of tables using the div 
print(f"There are {len(fatalities_divs)} tables in the Web page")



In [ ]:
# Counting number of table within each <div>
for divs in fatalities_divs:
    div_tbl=divs.find_all('table')
    print(len(div_tbl))
# The result confirms that each div tag contains 1 table to be scarped.

#### Extracting Table 1

In [ ]:
# Extracting the first table html by searching for table within the first div tag
table1=fatalities_divs[0].find('table')
table1

In [ ]:
# Extracting the Table Title within the thead 
table1_head_tr=table1.thead.find_all('tr')
# Title is within the first <tr> tag of thead
table1_title=table1_head_tr[0].getText().strip("\n")
print(f"The Table1 Title is :  {table1_title}")

In [ ]:
# The table1 headers are within the second <tr> element in the thead. 
# Within this, each column header is in the <th> element
table1_headers=[head_elem1.getText() for head_elem1 in table1_head_tr[1].find_all('th')]
print(f"The Table1 Headers are :  {table1_headers}")


In [ ]:
# Extracting the table data by searching the tbody and finding the <tr> element
table1_data_tr=table1.tbody.find_all('tr')
# Within each <tr> element, the table data is in both <td> and <th> elements, hence extracting both 
table1_data=[[data_elem2.getText() for data_elem2 in table1_data_tr[itera].find_all(['td','th'])] for itera in range(len(table1_data_tr)) ]   
table1_data

In [ ]:
# Creating the first table dataframe from the header and data extracted
table1_df=pd.DataFrame(table1_data,columns=table1_headers)

print(f"The Table1 dataframe is : {table1_title}")
display(table1_df.head())

### Extracting Table 2

In [ ]:
# Extracting the Second table html by searching for table within the second div tag
table2=fatalities_divs[1].find('table')
table2

In [ ]:
# Extracting the Table Title within the thead 
table2_head_tr=table2.thead.find_all('tr')
# Title is within the first <tr> tag of thead
table2_title=table2_head_tr[0].getText().strip("\n")
print(f"The Table2 Title is :  {table2_title}")


In [ ]:
# The table2 headers are within the second <tr> element in the thead. 
# Within this, each column header is in the <th> element
table2_headers=[head_elem1.getText() for head_elem1 in table2_head_tr[1].find_all('th')]
print(f"The Table2 Headers are :  {table2_headers}")


In [ ]:
# Extracting the table data by searching the tbody and finding the <tr> element
table2_data_tr=table2.tbody.find_all('tr')
# Within each <tr> element, the table data is in both <td> and <th> elements, hence extracting both 
table2_data=[[data_elem2.getText() for data_elem2 in table2_data_tr[itera].find_all(['td','th'])] for itera in range(len(table2_data_tr)) ]   


In [ ]:
# The Table2 data contains the number and percentage values. As we are only interested in the number,
# Skipping the percent values, by skipping the values at even index with the exception of 0 index.
table2_data_mod=[ [item for index,item in enumerate(table2_data[iterat]) if ( (index==0) | (index%2==1) )] for iterat in range(len(table2_data))]
# Printing top 4 elements of Modified Table2 data.
table2_data_mod[:4]

In [ ]:
# Creating the second table dataframe from the header and data extracted
table2_df=pd.DataFrame(table2_data_mod,columns=table2_headers)
print(f"The Table2 dataframe is : {table2_title}")
display(table2_df.head())

### 2. Extracting all tables in the HTML Source using Functions

##### As all the tables have similar structure as Table1 and Table2, using functions to extract the tables

In [ ]:
def create_table(table_data, skip=0):
    """
    This function returns the modified table data based on the skip indicator value passed. 
    For the first table, the skip value is 0 so the data is unmodified. For all other tables, the
    percentage values are skipped by passing the skip indicator value as 1.
    """
    # Creating a new variable to use in the Mod function
    modulo=skip+1
    # This will return modified table by skipping the even values(with the exception of 0 index)
    # If the Skip value is 0, the returned data is unmodified.
    #If the skip value is 1, the returned dataframe skips the values at even indexes.
    table_data_mod=[ [item for index,item in enumerate(table_data[iterat]) if ( (index==0) | (index%modulo==skip) )] for iterat in range(len(table_data))]
    return table_data_mod


In [ ]:
def create_DataFrame(final_table,table_headers):
    """
    This function will create final Dataframe based on the Table data and the header data passed as arguments
    """
    table_df=pd.DataFrame(final_table,columns=table_headers)
    return table_df

In [ ]:
def parse_HTML(index2):
    """
    This function will create the Final Dataframe by extracting the table data from Source HTML. 
    It takes the table number/index as an input and returns the dataframe. 
    For instance to extract the first table, the index passed will be 0.
    """
    # Parsing each div object created in the previous step
    for index1,item1 in enumerate(fatalities_divs):
        # By comparing the index in div object and the index passed in the function, 
        # we are making sure, only the desired table is created for the specified index.
        if index1==index2:
            # Searching for 'table' element in the div object
            table=fatalities_divs[index1].find('table')
            # Searching for tr element in the thead
            table_head_tr=table.thead.find_all('tr')
            # Table Title is within the first <tr> tag of thead
            table_title=table_head_tr[0].getText().strip("\n")
            # The table2 headers are within the second <tr> element in the thead.
            # within this, each column header is in the <th> element
            table_headers=[head_elem1.getText() for head_elem1 in table_head_tr[1].find_all('th')]
            # Extracting the table data by searching the tbody and finding the <tr> element
            table_data_tr=table.tbody.find_all('tr')
            # Within each <tr> element, the table data is in both <td> and <th> elements, hence extracting both 
            table_data=[[data_elem2.getText() for data_elem2 in table_data_tr[itera].find_all(['td','th'])] for itera in range(len(table_data_tr)) ]   
            # As some tables have percent and Number values, creating skip variable for those tables 
            # to skip the percentage elements. For the first table there is no skip and for all other table we skip the percent values
            skip=0 if index1==0 else 1
            """
            if index1==0:
                skip=0
            else:
                skip=1
                """
            # Calling the create table function to create modified table based on skip values passed
            final_table=create_table(table_data, skip)
            # Creating final dataframe based on the Table data and Header extracted
            final_df=create_DataFrame(final_table,table_headers)
        else:
            # If the specified index does not match, control is passed to the next index in the loop
            continue
        # Final dataframe is returned
        return final_df
    
    

##### Creating tables using dynamic variables

In [ ]:
# Getting the number of tables in the <div> elements
nbr_tables=len(fatalities_divs)
# Looping through each div element to create the dataframe by parsing the HTML
for index2 in range(nbr_tables):
    # Creating tables using dynamic variable in the format table_df_<index number> starting from 0 through 5
    globals()[f"table_df_{index2}"]=parse_HTML(index2)

In [ ]:
# testing the results by checking the tables extracted
table_df_0.head()

In [ ]:
# Getting top 5 elements of table 2
table_df_1.head()

In [ ]:
# Getting top 5 elements of table 3
table_df_2.head()

In [ ]:
# Getting top 5 elements of table 4
table_df_3.head()

In [ ]:
# Getting top 5 elements of table 6. 
table_df_5.head()
# Note: Table 5 is not displayed, as we won't be using that data for this project.

### 3. Data Cleansing and Transformations

#### Transformation 1: Creating new Dataframes by selecting the required columns

In [ ]:
# Creating 5 new dataframes with the required columns
fatal_crash_totals_df=table_df_0[["State","Population","Fatal crashes","Deaths"]]
deaths_by_roaduser_df=table_df_1
crash_types_df=table_df_2
alc_df=table_df_3[["State","Total drivers killed"]]
rural_urban_df=table_df_5

In [ ]:
# Assigning Name attribute for the Dataframes
fatal_crash_totals_df.name='fatal_crash_totals_df'
deaths_by_roaduser_df.name='deaths_by_roaduser_df'
crash_types_df.name='crash_types_df'
alc_df.name='alc_df'
rural_urban_df.name='rural_urban_df'

#### Transformation 2: Renaming columns

In [ ]:
# Renaming columns in fatal_crash_totals_df and doing an inplace update
fatal_crash_totals_df.rename(columns={"Fatal crashes":"Fatal_crashes"},inplace=True)

In [ ]:
# Renaming columns in deaths_by_roaduser_df and doing an inplace update
deaths_by_roaduser_df.rename(columns={"Car occupants":"Car_occup",
                                      "Pickup and SUV occupants":"Pickup_SUV_occup",
                                      "Large truck occupants":"Truck_occup",
                                      "Total*":"Total"
                                     },inplace=True)

In [ ]:
# Renaming columns in deaths_by_roaduser_df and doing an inplace update
crash_types_df.rename(columns={"Single-vehicle":"Single_vehicle",
                                      "Multiple-vehicle":"Multiple_vehicle",
                                      "All crashes":"All_crashes"
                                      },inplace=True)

In [ ]:
# Renaming columns in alcohol df and doing an inplace update
alc_df.rename(columns={"Total drivers killed":"Total_drivers_killed"},inplace=True)

#### Transformation 3: Converting data to Upper case

In [ ]:
def toUpper(df,col):
    """
    This fnction converts all the values in the given column of dataframe to Upper case and removes extra spaces from the string..
    Inputs: Dataframe name and Column Name.
    Performs an inplace update by converting to Upper case and removing extra spaces.
    """
    try:
        # Using apply method to convert to UpperCase and strip spaces.
        df[col]=df[col].apply(str.strip)
        df[col]=df[col].apply(str.upper)
    except TypeError:
        pass

In [ ]:
# Creating list of Dataframes, String columns from each Dataframe
all_df_list=[fatal_crash_totals_df,deaths_by_roaduser_df,crash_types_df,alc_df,rural_urban_df]
string_cols_list=[["State"],["State"],["State"],["State"],["State"]]

In [ ]:
# Looping through the list to convert the columns in string_cols_list from each Dataframe using the Index
for index3,df_name in enumerate(all_df_list):
    # Comparing based on index of the all_df_list and extracting the column from string_cols_list based on the index
    for col in string_cols_list[index3]:
        # converting the column data to Upper case
        toUpper(df_name,col)

#### Transformation 4: Remove comma seperator and convert to Integer

In [ ]:
def cleanse_int_cols(df,col):
    """
    This function will do an inplace update by removing comma. This is used to convert the columns that
    has Numeric data that are stored as Strings.
    It takes Dataframe name and column as inputs and returns a clean dataframe.
    """
    df[col]=df[col].str.replace(",","").astype(int)

In [ ]:
# Creating a list  of Integer columns
int_cols_list=[["Population","Fatal_crashes","Deaths"],["Car_occup","Pickup_SUV_occup","Truck_occup","Motorcyclists","Pedestrians","Bicyclists","Total"],
               ["Single_vehicle","Multiple_vehicle","All_crashes"],
               ["Total_drivers_killed"],
               ["Urban","Rural","Unknown","Total"]
              ]

In [ ]:
# Looping through the list to convert the columns in int_cols_list from each Dataframe using the Index values
for index4,df_name in enumerate(all_df_list):
    # Comparing based on index of the all_df_list and extracting the column from int_cols_list based on the index
    for col in int_cols_list[index4]:
        # calling Function to convert the string values to integer by removing commas in the number.
        cleanse_int_cols(df_name,col)
    

#### Transformation 5: Checking for Nulls

In [ ]:
def check_NaN(df,col):
    """
    This function checks the number of nulls in a columnof dataframe.
    Input: Takes the name of dataframe and the column name
    Returns: The count of nulls in the column
    """
    col_nan=df[col].isnull().sum()
    return col_nan


In [ ]:
def fill_NaN(df,col,fill_value=0):
    """
    This function calls the check_NaN to verify the presence of Nulls. If present, they are filled with  filler.
    Input: Takes, Dataframe name, Column name and the Filler value as Inputs.
    Performs an inplace update of filling Nulls with Filler.
    Also Prints the number of nulls present in the column.    
    """
    # Calling the check_NaN function to verify if nulls are present
    col_nan= check_NaN(df,col)
    # If Nulls are present, they are replaced with Fillers
    if col_nan >0:
        print(f"There are {col_nan} NaN values in the {col} column ")
        # Inplace update is performed by replacing NaN with filler and results are printed
        df[col].fillna(fill_value,inplace=True)
        print(f"Nulls are replaced with {fill_value} in the {col} column ")
    # If nulls are not present, the result is just printed.
    else:
        print(f"There are NO NaN values in the {col} column ")

In [ ]:
# Checking for Null values in all 5 dataframes by looping through the list
for df in all_df_list:
    print("**"*50)
    # Displaying the name of the Dataframe
    print(f"Checking for Nulls in {df.name}")
    # Looping through each column in the Dataframe
    for col in df.columns:
        fill_NaN(df,col,0)

#### Transformation 6: Merging Dataframes into 1

In [ ]:
# Merging Dataframes 1, 2 and 3 into a temp df
merged_temp_df1=fatal_crash_totals_df.merge(deaths_by_roaduser_df,on=['State'],how='inner').merge(crash_types_df,on=['State'],how='inner')
# Merging temp df with Dataframes 4 and 5
merged_fatal_df=merged_temp_df1.merge(alc_df,on=['State'],how='inner').merge(rural_urban_df,on=['State'],how='inner')


In [ ]:
# Printing top 5 rows of Merged Dataframe
merged_fatal_df.head()

#### Transformation 7: Drop redundant columns, Outliers and Rename other columns in Merged Dataframe

In [ ]:
# Dropping the duplicate columns that represent the same data such as as 'Total Deaths'
# Unknown values are considered Outliers and they are excluded
merged_fatal_df=merged_fatal_df.drop(['Total_x','All_crashes','Total_y','Unknown'],axis=1)
# Renaming columns in the Merged Dataframe
merged_fatal_df.rename(columns={"Deaths":"Total_Fatalities",
                                    "Car_occup":"Car_Fatals",
                                    "Pickup_SUV_occup":"SUV_Fatals",
                                    "Truck_occup":"Truck_Fatals",
                                    "Motorcyclists":"Motorcyclist_Fatals",
                                    "Pedestrians":"Pedestrian_Fatals",
                                    "Bicyclists":"Bicyclist_Fatals",
                                    "Single-vehicle":"Single_Vehicle_Fatals",
                                    "Multiple-vehicle":"Multiple_Vehicle_Fatals",
                                    "Total_drivers_killed":"Alcohol_Related_Fatals",
                                    "Urban":"Urban_Fatals",
                                    "Rural":"Rural_Fatals"
                                      },inplace=True)

In [ ]:
# Printing top 5 rows of Merged Dataframe
merged_fatal_df.head()

#### Transformation 8: Adding New columns 

In [ ]:
# Importing the data about 50 US states and DC including the State code, Region and Division from dictionary
US_States_dict={'State': {'Alaska': 'Alaska',
  'Alabama': 'Alabama',
  'Arkansas': 'Arkansas',
  'Arizona': 'Arizona',
  'California': 'California',
  'Colorado': 'Colorado',
  'Connecticut': 'Connecticut',
  'District of Columbia': 'District of Columbia',
  'Delaware': 'Delaware',
  'Florida': 'Florida',
  'Georgia': 'Georgia',
  'Hawaii': 'Hawaii',
  'Iowa': 'Iowa',
  'Idaho': 'Idaho',
  'Illinois': 'Illinois',
  'Indiana': 'Indiana',
  'Kansas': 'Kansas',
  'Kentucky': 'Kentucky',
  'Louisiana': 'Louisiana',
  'Massachusetts': 'Massachusetts',
  'Maryland': 'Maryland',
  'Maine': 'Maine',
  'Michigan': 'Michigan',
  'Minnesota': 'Minnesota',
  'Missouri': 'Missouri',
  'Mississippi': 'Mississippi',
  'Montana': 'Montana',
  'North Carolina': 'North Carolina',
  'North Dakota': 'North Dakota',
  'Nebraska': 'Nebraska',
  'New Hampshire': 'New Hampshire',
  'New Jersey': 'New Jersey',
  'New Mexico': 'New Mexico',
  'Nevada': 'Nevada',
  'New York': 'New York',
  'Ohio': 'Ohio',
  'Oklahoma': 'Oklahoma',
  'Oregon': 'Oregon',
  'Pennsylvania': 'Pennsylvania',
  'Rhode Island': 'Rhode Island',
  'South Carolina': 'South Carolina',
  'South Dakota': 'South Dakota',
  'Tennessee': 'Tennessee',
  'Texas': 'Texas',
  'Utah': 'Utah',
  'Virginia': 'Virginia',
  'Vermont': 'Vermont',
  'Washington': 'Washington',
  'Wisconsin': 'Wisconsin',
  'West Virginia': 'West Virginia',
  'Wyoming': 'Wyoming'},
 'State Code': {'Alaska': 'AK',
  'Alabama': 'AL',
  'Arkansas': 'AR',
  'Arizona': 'AZ',
  'California': 'CA',
  'Colorado': 'CO',
  'Connecticut': 'CT',
  'District of Columbia': 'DC',
  'Delaware': 'DE',
  'Florida': 'FL',
  'Georgia': 'GA',
  'Hawaii': 'HI',
  'Iowa': 'IA',
  'Idaho': 'ID',
  'Illinois': 'IL',
  'Indiana': 'IN',
  'Kansas': 'KS',
  'Kentucky': 'KY',
  'Louisiana': 'LA',
  'Massachusetts': 'MA',
  'Maryland': 'MD',
  'Maine': 'ME',
  'Michigan': 'MI',
  'Minnesota': 'MN',
  'Missouri': 'MO',
  'Mississippi': 'MS',
  'Montana': 'MT',
  'North Carolina': 'NC',
  'North Dakota': 'ND',
  'Nebraska': 'NE',
  'New Hampshire': 'NH',
  'New Jersey': 'NJ',
  'New Mexico': 'NM',
  'Nevada': 'NV',
  'New York': 'NY',
  'Ohio': 'OH',
  'Oklahoma': 'OK',
  'Oregon': 'OR',
  'Pennsylvania': 'PA',
  'Rhode Island': 'RI',
  'South Carolina': 'SC',
  'South Dakota': 'SD',
  'Tennessee': 'TN',
  'Texas': 'TX',
  'Utah': 'UT',
  'Virginia': 'VA',
  'Vermont': 'VT',
  'Washington': 'WA',
  'Wisconsin': 'WI',
  'West Virginia': 'WV',
  'Wyoming': 'WY'},
 'Region': {'Alaska': 'West',
  'Alabama': 'South',
  'Arkansas': 'South',
  'Arizona': 'West',
  'California': 'West',
  'Colorado': 'West',
  'Connecticut': 'Northeast',
  'District of Columbia': 'South',
  'Delaware': 'South',
  'Florida': 'South',
  'Georgia': 'South',
  'Hawaii': 'West',
  'Iowa': 'Midwest',
  'Idaho': 'West',
  'Illinois': 'Midwest',
  'Indiana': 'Midwest',
  'Kansas': 'Midwest',
  'Kentucky': 'South',
  'Louisiana': 'South',
  'Massachusetts': 'Northeast',
  'Maryland': 'South',
  'Maine': 'Northeast',
  'Michigan': 'Midwest',
  'Minnesota': 'Midwest',
  'Missouri': 'Midwest',
  'Mississippi': 'South',
  'Montana': 'West',
  'North Carolina': 'South',
  'North Dakota': 'Midwest',
  'Nebraska': 'Midwest',
  'New Hampshire': 'Northeast',
  'New Jersey': 'Northeast',
  'New Mexico': 'West',
  'Nevada': 'West',
  'New York': 'Northeast',
  'Ohio': 'Midwest',
  'Oklahoma': 'South',
  'Oregon': 'West',
  'Pennsylvania': 'Northeast',
  'Rhode Island': 'Northeast',
  'South Carolina': 'South',
  'South Dakota': 'Midwest',
  'Tennessee': 'South',
  'Texas': 'South',
  'Utah': 'West',
  'Virginia': 'South',
  'Vermont': 'Northeast',
  'Washington': 'West',
  'Wisconsin': 'Midwest',
  'West Virginia': 'South',
  'Wyoming': 'West'},
 'Division': {'Alaska': 'Pacific',
  'Alabama': 'East South Central',
  'Arkansas': 'West South Central',
  'Arizona': 'Mountain',
  'California': 'Pacific',
  'Colorado': 'Mountain',
  'Connecticut': 'New England',
  'District of Columbia': 'South Atlantic',
  'Delaware': 'South Atlantic',
  'Florida': 'South Atlantic',
  'Georgia': 'South Atlantic',
  'Hawaii': 'Pacific',
  'Iowa': 'West North Central',
  'Idaho': 'Mountain',
  'Illinois': 'East North Central',
  'Indiana': 'East North Central',
  'Kansas': 'West North Central',
  'Kentucky': 'East South Central',
  'Louisiana': 'West South Central',
  'Massachusetts': 'New England',
  'Maryland': 'South Atlantic',
  'Maine': 'New England',
  'Michigan': 'East North Central',
  'Minnesota': 'West North Central',
  'Missouri': 'West North Central',
  'Mississippi': 'East South Central',
  'Montana': 'Mountain',
  'North Carolina': 'South Atlantic',
  'North Dakota': 'West North Central',
  'Nebraska': 'West North Central',
  'New Hampshire': 'New England',
  'New Jersey': 'Middle Atlantic',
  'New Mexico': 'Mountain',
  'Nevada': 'Mountain',
  'New York': 'Middle Atlantic',
  'Ohio': 'East North Central',
  'Oklahoma': 'West South Central',
  'Oregon': 'Pacific',
  'Pennsylvania': 'Middle Atlantic',
  'Rhode Island': 'New England',
  'South Carolina': 'South Atlantic',
  'South Dakota': 'West North Central',
  'Tennessee': 'East South Central',
  'Texas': 'West South Central',
  'Utah': 'Mountain',
  'Virginia': 'South Atlantic',
  'Vermont': 'New England',
  'Washington': 'Pacific',
  'Wisconsin': 'East North Central',
  'West Virginia': 'South Atlantic',
  'Wyoming': 'Mountain'}}

In [ ]:
# Creating dataframe from the US states dictionary
US_State_Region_df=pd.DataFrame(US_States_dict)
US_State_Region_df.head()

In [ ]:
# Converting the data in US states Dataframe to Upper Case
for col in US_State_Region_df.columns:
    toUpper(US_State_Region_df,col)

In [ ]:
# Doing a Left join between merged dataframe and US Stataes DF to add columns such as Region, Devision, State code
final_fatals_df=merged_fatal_df.merge(US_State_Region_df,on=['State'],how='left')
# Converting the Column names of final dataframe to Upper Case
final_fatals_df.columns=final_fatals_df.columns.str.upper()
# Renaming column
final_fatals_df.rename(columns={"STATE CODE":"STATE_CODE"},inplace=True)

In [ ]:
# Treating for Nulls in the final dataframe
for col in final_fatals_df.columns:
    fill_NaN(final_fatals_df,col,'NA')

#### Transformation 9: GroupBy and Sort Transformations

In [ ]:
# Creating a dataframe of 50 states and the DC excluding the US Totals
fatals_USStates_df=final_fatals_df[~final_fatals_df['STATE_CODE'].str.contains('NA')]


In [ ]:
# Grouping the Datafarme based on Division and calculating the sum of Fatalities in each division. 
# Results are ordered in descending order
fatals_byDivision_df=fatals_USStates_df.groupby(["DIVISION"],sort=True)["TOTAL_FATALITIES"].sum().sort_values(ascending=False)
fatals_byDivision_df

In [ ]:
# Grouping the Dataframe based on Region and getting the sum of Alcohol related deaths in each region
# Results are ordered in descending order
alc_fatals_byRegion=fatals_USStates_df.groupby(["REGION"],sort=True)["ALCOHOL_RELATED_FATALS"].sum().sort_values(ascending=False)
alc_fatals_byRegion

#### Transformation 10: Stack and Unstack Transformations

In [ ]:
# Creating a Dataframe subset and sortinng results based on Region name and Fatalities in Urban
urban_rural_df1=fatals_USStates_df[["STATE","URBAN_FATALS","RURAL_FATALS","REGION"]].sort_values(by=['REGION',"URBAN_FATALS"],ascending=False)
# Selecting only Midwest Region
urban_rural_df1=urban_rural_df1[urban_rural_df1["REGION"].isin(['MIDWEST'])]
# Stting State name as Index
urban_rural_df1.set_index("STATE",inplace=True)
urban_rural_df1

In [ ]:
# Calling Unstack method to stack the Region, Urban and Rural fatalities based on State name
display(urban_rural_df1.unstack())

In [ ]:
# Reindexing the dataframe and giving a name for the columns to be used in the next step
urban_rural_df1=urban_rural_df1.reindex(columns=["URBAN_FATALS","RURAL_FATALS","REGION"])
urban_rural_df1.columns.name="FATAL_STATS"

In [ ]:
# Using Stack operation to show the results grouped by each State
urban_rural_expand_df1=urban_rural_df1.stack().reset_index().rename(columns={0:"VALUE"})
urban_rural_expand_df1.head(21)

#### Transformation 11: Pivot & Melt Transformations

In [ ]:
# Using Pivot operation on the Stacked dataframe to restore the original data
urban_rural_pivot_df1=urban_rural_expand_df1.pivot(index="STATE",columns="FATAL_STATS",values="VALUE")
urban_rural_pivot_df1

In [ ]:
# Creating a new Subset dataframe to be used for Melt function
urban_rural_df2=fatals_USStates_df[["STATE","URBAN_FATALS","RURAL_FATALS","DIVISION"]].sort_values(by=['DIVISION',"URBAN_FATALS"],ascending=False)
# Using the Mountain Division data only
urban_rural_df2=urban_rural_df2[urban_rural_df2["DIVISION"].isin(['MOUNTAIN'])]

In [ ]:
# Using the Melt function to group the data based on State and display the Division name,
# Urban and Rural fatalities in that state. This is similar to the stack operation done in above step.
urban_rural_melt_df2=pd.melt(urban_rural_df2,id_vars="STATE").sort_values(by=['STATE'])
urban_rural_melt_df2

In [ ]:
# Using Pivot on the above Melt step to restore the original data
urban_rural_pivot_df2=urban_rural_melt_df2.pivot(index="STATE",columns="variable",values="value")
urban_rural_pivot_df2

### 4. Printing the Final dataframe

In [ ]:
display(fatals_USStates_df)

### 5.Ethical Implications

##### a. While reporting the Alcohol involvement, not all data is reported. Some states have a high reporting rate such as Hawaii, while some states like Mississippi have very low reporting rate. This may lead to inconsitencies while deriving metrics and conclusions from the data.
##### b. While extracting the Rural and Urban data from the table , the  "Unknown" category was excluded from this analysis  and was considered an Outlier. Care must be taken while reporting the results based on this category alone. For instance, in Mississippi there were about 79 Unknown cases which was rougly 10% of the data. Hence, such Unknwn data may lead to misleading results.
##### c. The data about the seat belt use(Restraint use) was not included in this study as there were many unknown values in this category for many states. However some states such as California had the highest percent of seat belt use while New-Hamshire had the lowest. Hence the data from this table can be useful while performing Analysis on certain states where we have good amount of data. Hence though this data was not considered significant for this project, it can be useful while analysing the fatalities data for certain states.

In [ ]:
fatals_USStates_df.to_excel('formatted_webdata_df.xlsx')

## 